In [30]:
import os
import sqlite3
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import datetime

In [43]:
# Initialize database
def init_db():
    conn = sqlite3.connect('language_learning.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS mistakes
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  timestamp TEXT,
                  user_input TEXT,
                  correction TEXT,
                  category TEXT)''')
    conn.commit()
    conn.close()

In [44]:
def log_mistake(user_input, correction, category):
    """Log language learning mistakes to SQLite database"""
    try:
        # Use the existing database connection you've already set up
        conn = sqlite3.connect('language_learning.db')
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO mistakes 
            (timestamp, user_input, correction, category)
            VALUES (?, ?, ?, ?)
        ''', (datetime.datetime.now(), user_input, correction, category))
        
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"Error logging mistake: {e}")

In [45]:
# Generate mistake report
def generate_report():
    conn = sqlite3.connect('language_learning.db')
    c = conn.cursor()
    c.execute("SELECT * FROM mistakes")
    mistakes = c.fetchall()
    
    print("\n--- Learning Report ---")
    for mistake in mistakes:
        print(f"\nMistake {mistake[0]}:")
        print(f"Original: {mistake[2]}")
        print(f"Corrected: {mistake[3]}")
        print(f"Type: {mistake[4]}")
    conn.close()

In [46]:
class LanguageTutor:
    def __init__(self, target_lang, known_lang, proficiency):
        # Initialize LLM 
        self.llm = ChatGroq(
            temperature=0.7,
            model_name="llama3-8b-8192",
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
       
        # Initialize memory 
        self.message_history = []
       
        self.scenes = {
            '1': 'restaurant',
            '2': 'airport',
            '3': 'hotel',
            '4': 'shop'
        }
        
        # Store language learning context
        self.target_lang = target_lang
        self.known_lang = known_lang
        self.proficiency = proficiency
        self.current_scene = None
        
        # Create prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a {target_lang} language tutor.
            The student knows {known_lang} and is at {proficiency} level.
            Current scenario: {current_scene}.
            Conduct conversation in {target_lang}.
            Correct mistakes in this format:
            Correction: [explanation]
            Response: [actual response]"""),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}")
        ])
       
        # Create runnable
        self.runnable = self.prompt | self.llm
        

    def select_scene(self):
        """Select conversation scenario and start the conversation"""
        print("\nSelect a scenario:")
        for key, value in self.scenes.items():
            print(f"{key}. {value.capitalize()}")
       
        choice = input("Enter choice (1-4): ")
        self.current_scene = self.scenes.get(choice, 'restaurant')
       
        # Clear previous message history
        self.message_history = []
       
        # Prepare context for initial greeting
        context = {
            "target_lang": self.target_lang,
            "known_lang": self.known_lang,
            "proficiency": self.proficiency,
            "current_scene": self.current_scene,
            "history": self.message_history,
            "input": "Greet the student and start the conversation"
        }
       
        # Invoke the runnable
        response = self.runnable.invoke(context)
        return response.content


    def chat_loop(self):
        """Main conversation loop"""
        print("\nStart chatting (type 'exit' to end):")
        while True:
            user_input = input("\nYou: ")
            if user_input.lower() == 'exit':
                break
           
            # Add user message to history
            self.message_history.append(HumanMessage(content=user_input))
           
            # Prepare context for response
            context = {
                "target_lang": self.target_lang,
                "known_lang": self.known_lang,
                "proficiency": self.proficiency,
                "current_scene": self.current_scene,
                "history": self.message_history,
                "input": user_input
            }
           
            # Get bot response
            response = self.runnable.invoke(context)
            bot_response = response.content
           
            if "Correction:" in bot_response:
                parts = bot_response.split("Response:")
                correction_part = parts[0].replace("Correction:", "").strip()
                response_part = parts[1].strip() if len(parts) > 1 else ""
               
                # Log the mistake
                log_mistake(user_input, correction_part, "Grammar")
                
                print(f"\nBot: {response_part}")
               
                # Add assistant message to history
                self.message_history.append(AIMessage(content=response_part))
            else:
                print(f"\nBot: {bot_response}")
                # Add assistant message to history
                self.message_history.append(AIMessage(content=bot_response))

In [47]:
def main():
    init_db()
    # Collect user preferences
    target_lang = input("What language do you want to learn? ")
    known_lang = input("What is your native language? ")
    proficiency = input("What's your current proficiency level (beginner/intermediate/advanced)? ")

    # Create and run the language tutor
    tutor = LanguageTutor(target_lang, known_lang, proficiency)
    print(tutor.select_scene())
    tutor.chat_loop()
    generate_report()

if __name__ == "__main__":
    main()


Select a scenario:
1. Restaurant
2. Airport
3. Hotel
4. Shop
Hello! Welcome to the restaurant. How are you today?

(Note: I'll respond in English and correct any mistakes the student makes as we go along.)

Start chatting (type 'exit' to end):

Bot: 

Bot: Ah, there are many good restaurants around here. What kind of cuisine are you in the mood for?

Bot: There's a great South Indian restaurant just down the street. It's called "Tamil Nadu". They have a wide variety of dosas, idlis, and vadas. Would you like to try it?

Bot: They have a combo special for idlis and sambar. It's only about $5. Would you like to try that?

Bot: I'll go ahead and order it for you. Would you like to drink anything with that, like a cup of coffee or tea?

Bot: I'll add that to your order. So, to confirm, I've ordered the idlis and sambar combo, and a cup of filter coffee, is that correct?

Bot: You're welcome! I'll go ahead and bring your order to you.

Bot: Bye!


KeyboardInterrupt: Interrupted by user

In [48]:
generate_report()


--- Learning Report ---

Mistake 1:
Original: hello how are you
Corrected: I'm doing well, thank you! How about you?

 [None]
Type: Grammar

Mistake 2:
Original: which restaurent is good here
Corrected: "restaurant" should be written as one word, "restaurant".
Type: Grammar
